In [72]:
import numpy as np
import pandas as pd
import os
import ast
import timeit

In [73]:
path: str = './data/'
prokaryotes: pd.DataFrame = pd.read_csv(os.path.join(path, "prokaryotes_sec_structure.csv"))
prokaryotes = prokaryotes[prokaryotes['run_name'].str.contains('lysate', case=True)].dropna(subset=['meltPoint'])
secs = ['Helix1','Turn1','Sheet1','Helix2','Sheet2']
if isinstance(prokaryotes.iloc[0,28],str):
    for s in secs:
        prokaryotes[s] = prokaryotes[s].apply(lambda x: ast.literal_eval(x) if pd.isnull(x)==False else x)
prokaryotes.reset_index(drop=True, inplace=True)

In [74]:
def oligo_motif(data, oligolength):
    from function import p_val
    blacklist = np.array([])
    for n in range(len(data)-oligolength-1):    # 4 ersetzen mit oligolänge input -1
        oligo = data[n:n+oligolength]           # 5 ersetzen mit oligolänge input
        if oligo not in blacklist:
            np.append(blacklist,oligo)
            oligocorr = prokaryotes['Sequence'].apply(lambda x: x.count(oligo)).corr(prokaryotes['meltPoint'])
            if oligocorr > 0.3 and p_val(oligocorr, prokaryotes['Sequence'].apply(lambda x: x.count(oligo)).sum(), 0.05)[1] == True:
                prokaryotes[f'{oligo}motif'] = prokaryotes['Sequence'].apply(lambda x: x.count(oligo))  # prokaryotes ersetzen mit input df, 'Sequence' ersetzen mit input Sequence column
                prokaryotes[f'{oligo}motif'] = np.array(prokaryotes[f'{oligo}motif'])/np.array(prokaryotes['Length'])   # Length ersetzen mit input Length column

In [75]:
def oligo_motifsub(prot, oligolength,blacklist):
    from function import p_val
    for n in range(len(prot)-oligolength-1):    
        oligo = prot[n:n+oligolength]          
        if oligo not in blacklist:
            np.append(blacklist,oligo)
            oligocorr = prokaryotes['Sequence'].apply(lambda x: x.count(oligo)).corr(prokaryotes['meltPoint'])
            if oligocorr > 0.2 and p_val(oligocorr, prokaryotes['Sequence'].apply(lambda x: x.count(oligo)).sum(), 0.05)[1] == True:
                prokaryotes[f'{oligo}motif'] = prokaryotes['Sequence'].apply(lambda x: x.count(oligo))  
                prokaryotes[f'{oligo}motif'] = np.array(prokaryotes[f'{oligo}motif'])/np.array(prokaryotes['Length'])

In [76]:
def oligo_motif(data, oligolength):
    blacklist = np.array([])
    prokaryotes['Sequence'].apply(lambda x: oligo_motifsub(x,oligolength,blacklist))

prokaryotes['Testoligo'] = prokaryotes['Sequence'].apply(lambda x: x.count('TAT'))
prokaryotes['Testoligo'].sum()

testoligo = 'TAT'
prokaryotes['Sequence'].apply(lambda x: x.count(testoligo)).corr(prokaryotes['meltPoint'])

In [77]:
testdf = prokaryotes.loc[0:99,:].reset_index(drop=True)

In [78]:
'''df = prokaryotes
oligolength = 4
from function import p_val
bl = []
for n in range(len(df)):
    print(n)
    for a in range(len(df.loc[n,'Sequence'])-oligolength-1):
        oligo = df.loc[n,'Sequence'][a:a+oligolength]
        if oligo not in bl:
            bl.append(oligo)
            oligocorr = df['Sequence'].apply(lambda x: x.count(oligo)).corr(df['meltPoint'])
            oligocountsum = df['Sequence'].apply(lambda x: x.count(oligo)).sum()
            if oligocorr > 0.2 and oligocountsum > 20 and p_val(oligocorr, len(df), 0.05)[1] == True:
                df[f'{oligo}motif'] = df['Sequence'].apply(lambda x: x.count(oligo))  
                #df[f'{oligo}motif'] = np.array(testdf[f'{oligo}motif'])/np.array(df['Length'])'''

"df = prokaryotes\noligolength = 4\nfrom function import p_val\nbl = []\nfor n in range(len(df)):\n    print(n)\n    for a in range(len(df.loc[n,'Sequence'])-oligolength-1):\n        oligo = df.loc[n,'Sequence'][a:a+oligolength]\n        if oligo not in bl:\n            bl.append(oligo)\n            oligocorr = df['Sequence'].apply(lambda x: x.count(oligo)).corr(df['meltPoint'])\n            oligocountsum = df['Sequence'].apply(lambda x: x.count(oligo)).sum()\n            if oligocorr > 0.2 and oligocountsum > 20 and p_val(oligocorr, len(df), 0.05)[1] == True:\n                df[f'{oligo}motif'] = df['Sequence'].apply(lambda x: x.count(oligo))  \n                #df[f'{oligo}motif'] = np.array(testdf[f'{oligo}motif'])/np.array(df['Length'])"

In [86]:
import pandas as pd
from function import p_val
oligolength = 4       #lengths of oligos to test, found for 3,4

sequences = prokaryotes['Sequence']
meltPoints = prokaryotes['meltPoint']

oligo_counts = {}

for seq in sequences:                               # Count oligos in sequences and assign count to dictionary
    for i in range(len(seq) - oligolength + 1):
        oligo = seq[i:i + oligolength]
        if oligo in oligo_counts:
            oligo_counts[oligo] += 1
        else:
            oligo_counts[oligo] = 1

filtered_oligos = {oligo: count for oligo, count in oligo_counts.items() if count > 20} # Filter oligos with count > 20

for oligo in filtered_oligos.keys():
    oligo_series = sequences.apply(lambda x: x.count(oligo))
    oligocorr = oligo_series.corr(meltPoints)
    if abs(oligocorr) > 0.2 and p_val(oligocorr, len(prokaryotes), 0.05)[1] == True:     #Filter only oligos with correlation > 0.2 and p-value < 0.05
        prokaryotes[f'{oligo}motif'] = oligo_series



In [80]:
arr = np.array([])
np.append(arr,'AAA')
if 'AAA' in arr:
    print('Y')

In [81]:
correl = prokaryotes.corr(numeric_only=True)['meltPoint']